In [1]:
from scipy.special import binom, comb
import math
import numpy as np

In [2]:
#Computations of Generator and Relation degrees for Gorenstein ideals (as in Diesel 1996)
def hilbert(q,p,t):
    #Given generator and relation degrees q and p, compute the Hilbert function in Degree d
    n = len(p)
    s = p[0]+q[0]
    output = binom(2+t,2)
    if 2+t-s>=2:
        output -= binom(2+t-s,2)
    for i in range(n):
        if 2+t-q[i] >=2:
            output -= binom(2+t-q[i],2)
        if 2+t - p[i] >=2:
            output += binom(2+t-p[i],2)
    return output

def min_gen(T):
    #Compute the minimal number of generators for a given hilbert function
    output = -1
    
    n = len(T)
    first_diff = [1]
    first_diff.extend([T[i+1]-T[i] for i in range(n-1)])
    first_diff.append(-1)
    
    second_diff = [1]
    second_diff.extend([first_diff[i+1]-first_diff[i] for i in range(n-1)])
    second_diff.append(-1)
    
    third_diff = [1]
    third_diff.extend([second_diff[i+1]-second_diff[i] for i in range(n-1)])
    third_diff.append(-1)
    
    total = sum([-d for d in third_diff if d < 0])
    output += 2*math.ceil(total/2)
    return  output

def generate_partitions(curr_a, remaining_length, max_val,partitions,k,d):
    #Enumerate all self-complementary partitions
    
    if remaining_length == 0:
        #make self-complementary
        curr_a.extend(np.zeros(k))
        for i in range(k):
            curr_a[2*k-(i+1)] = 2*d-2*k+2-curr_a[i]
        partitions.append(curr_a)
        return
    start = 0 if not curr_a else curr_a[-1]
    for i in range(start, max_val+1):
        generate_partitions(curr_a + [i],remaining_length - 1,max_val,partitions,k,d)
        
def reduce_r(r):
    #Reduce to minimal set of generators and relations by deleting (i,j) if r_i + r_j = 0
    
    indices_to_check = [idx for idx in range(len(r))]
    indices_to_keep = []
    while len(indices_to_check) >= 1:
        idx = indices_to_check[0]
        indices_to_check.remove(idx)
        keep_idx = 1
            
        for j in indices_to_check:
            if r[idx] + r[j] == 0:
                indices_to_check.remove(j)
                keep_idx = 0
                break
                
        if keep_idx ==  1:        
            indices_to_keep.append(idx) 
            
    return([r[idx] for idx in indices_to_keep])

def chopped(P,Q,d):
    #compute an upper bound on the dimension of <J_d>_{2d}, where J is the Gorenstein 
    #ideal with generator degrees Q and relation degrees P
    
    dim = 0
    Q = np.array(Q)
    P = np.array(P)
    
    for q in Q[np.array(Q)<= d]:
        dim += binom(2 + (2*d - q), 2)
    
    if max(Q) >= (d+1):
        Relation_Degree = min(Q[np.array(Q)>=(d+1)])
        for p in P[np.array(P)<Relation_Degree]:
            dim -= binom(2 + (2*d - p),2)
        
    return dim
    
    
def possible_ideals(d):
    #Compute generator and relation degrees of all Gorenstein ideals with socle degree 2d
    #satisfying dimension conditions in degree d
    
    Generators = []
    Relations = []
    blocks = []
    
    for k in range(3,d+1):
        partitions = []
        generate_partitions([],k,d-k+1,partitions,k,d)
    
        for a in partitions:
            r = [2*d - 2*k + 2 +1]
            r.extend([2*d - 2*k + 2 - 2*ai +1 for ai in a])
            
            r = reduce_r(r)
        
            Q = [0.5*(2*d + 3 - ri) for ri in r]
            P = [2*d + 3 - qi for qi in Q]
            
            Q = np.array(Q)
            
            low_degree = Q[Q<=d]
            
            potential = 0
            
            if len(low_degree) >=4:
                potential = 1
            
            if len(low_degree) == 3 and sum(low_degree)<=2*d + 3:
                potential = 1
                
        
            #only add if degree d part of ideal has large enough dimension
            if potential == 1 and hilbert(Q,P,d) <= binom(2+d,2) - (d+1) and hilbert(Q,P,d) >= 3*d - 2:                
                
                #only add if degree d part can generate hyperplane in 2d
                if chopped(P,Q,d) >= (binom(2+(2*d), 2) - 1):
                    Generators.append(Q)
                    #print("Low Degree")
                    #print(low_degree)
                    Relations.append(P)
                    blocks.append(a)

    return Generators
    

In [3]:
#Enumerate all generator degrees for Gorenstein ideals satisfying dimension conditions    
for d in range(4,7):
    print("Generator Degrees for d =",d)
    gen_list = possible_ideals(d)
    for Q in gen_list:
        print(Q)
    print("\n")

Generator Degrees for d = 4
[3. 4. 4.]


Generator Degrees for d = 5
[3. 5. 5.]
[4. 4. 5.]
[4. 5. 5. 5. 7.]


Generator Degrees for d = 6
[3. 6. 6.]
[ 4.  4.  6.  6. 10.]
[4. 5. 6.]
[4. 6. 6. 6. 8.]
[5. 5. 5.]
[5. 5. 6. 6. 8.]
[5. 6. 6. 6. 6. 8. 8.]


